<a href="https://colab.research.google.com/github/nishant-sheoran/Datascience/blob/main/DS_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment by:RA2211003011324 & RA2211003011325

##Beautiful Soup4 + JS

In [ ]:
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
!apt-get update
!apt-get install -y chromium-chromedriver
!pip install selenium


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Configure Selenium for JS-rendered pages
def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.binary_location = "/usr/bin/chromium-browser"  # Correct binary location

    try:
        driver = webdriver.Chrome(options=chrome_options)
        return driver
    except Exception as e:
        print(f"Error setting up Selenium WebDriver: {e}")
        return None

def scrape_articles(base_url, num_pages=5):
    articles = []
    driver = setup_driver()

    if not driver:
        print("Driver initialization failed. Exiting...")
        return pd.DataFrame(articles)

    for page in range(1, num_pages + 1):
        try:
            print(f"Scraping page {page}...")
            driver.get(f"{base_url}/page/{page}")
            time.sleep(3)  # Allow time for JS to load

            soup = BeautifulSoup(driver.page_source, "html.parser")
            news_items = soup.find_all("article")  # Adjust based on site structure

            if not news_items:
                print(f"No articles found on page {page}")
                continue

            for item in news_items:
                try:
                    headline = item.find("h2").text.strip() if item.find("h2") else "N/A"
                    author = item.find(class_="author").text.strip() if item.find(class_="author") else "Unknown"
                    date = item.find("time").text.strip() if item.find("time") else "Unknown"
                    content_link = item.find("a")["href"] if item.find("a") else None
                    category = item.find(class_="category").text.strip() if item.find(class_="category") else "General"

                    if content_link:
                        try:
                            driver.get(content_link)
                            time.sleep(2)
                            article_soup = BeautifulSoup(driver.page_source, "html.parser")
                            content = article_soup.find("div", class_="article-body").text.strip() if article_soup.find("div", class_="article-body") else "No content"
                        except Exception as e:
                            print(f"Error fetching article content: {e}")
                            content = "Error fetching content"
                    else:
                        content = "No content"

                    articles.append({
                        "Headline": headline,
                        "Author": author,
                        "Publication Date": date,
                        "Category": category,
                        "Content": content
                    })
                except Exception as e:
                    print(f"Error processing article: {e}")
        except Exception as e:
            print(f"Error on page {page}: {e}")

    driver.quit()
    return pd.DataFrame(articles)

# Example usage
news_url = "https://www.bbc.com/news"  # Change to the actual site
news_data = scrape_articles(news_url, num_pages=3)

# Save to CSV
if not news_data.empty:
    news_data.to_csv("news_articles.csv", index=False)
    print("Scraping completed. Data saved as news_articles.csv")
else:
    print("No data scraped. Exiting...")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping completed. Data saved as news_articles.csv


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.bbc.com/news"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

articles = []
for item in soup.find_all("div", class_="gs-c-promo"):
    headline = item.find("h3").text if item.find("h3") else None
    link = item.find("a")["href"] if item.find("a") else None
    articles.append({"headline": headline, "link": link})

print(articles)

[]


In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from collections import Counter
import re
import nltk
nltk.download('stopwords')

# Read the sample news CSV file
df = pd.read_csv("/content/sample_data/sample_news.csv", low_memory=False)

# Print column names for verification
print("Columns in dataset:", df.columns)

# Handle missing 'author' column values
if 'author' not in df.columns:
    df['author'] = "Unknown"
else:
    df["author"].fillna("Unknown", inplace=True)

# Fix encoding for 'content' column
if 'content' in df.columns:
    df["content"] = df["content"].astype(str).apply(lambda x: x.encode("utf-8", "ignore").decode("utf-8"))
else:
    raise ValueError("Column 'content' not found. Please check your CSV file.")

# Function to extract keywords
def extract_keywords(text):
    words = re.findall(r"\b\w+\b", text.lower())
    words = [word for word in words if word not in stopwords.words("english")]
    return Counter(words).most_common(10)

# Apply keyword extraction
df["keywords"] = df["content"].apply(extract_keywords)

# Save cleaned data
cleaned_csv_path = "/content/sample_data/sample_news.csv"
df.to_csv(cleaned_csv_path, index=False)

print("Data cleaning completed. File saved as 'cleaned_news.csv'.")


Columns in dataset: Index(['headline', 'author', 'publication_date', 'category', 'content',
       'source_url'],
      dtype='object')
Data cleaning completed. File saved as 'cleaned_news.csv'.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-13-64fcfc39134a>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["author"].fillna("Unknown", inplace=True)


In [ ]:
import sqlite3

conn = sqlite3.connect("news.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS articles (
    id INTEGER PRIMARY KEY,
    headline TEXT,
    author TEXT,
    publication_date TEXT,
    category TEXT,
    content TEXT,
    source_url TEXT
)
""")

# Insert data
df = pd.read_csv("/content/sample_data/sample_news.csv")
df.to_sql("articles", conn, if_exists="replace", index=False)

conn.commit()
conn.close()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", max_features=10)
tfidf_matrix = vectorizer.fit_transform(df["content"])
keywords = vectorizer.get_feature_names_out()

print("Top 10 Keywords:", keywords)


Top 10 Keywords: ['ai' 'major' 'markets' 'medical' 'nation' 'positive' 'rise' 'scientists'
 'sparked' 'stock']


In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_entities(text):
    doc = nlp(text)
    return [ent.text for ent in doc.ents if ent.label_ in ["ORG", "GPE", "PERSON"]]

df["entities"] = df["content"].apply(extract_entities)
